# **Proyecto Final - Lending Club**

## **Curso:** Análisis de grandes volúmenes de datos

## **Tecnológico de Monterrey**

## Predicción de riesgo crediticio y segmentación de clientes

## **Equipo :** 13
 **Integrantes :** 
- Kevin Balderas Sánchez – A01795149
- Alan Jasso Arenas – A01383272
- José Florencio Maguey Peralta – A01796727
- Oscar Luis Guadarrama Jiménez – A01796245

---


## **Introducción**

Lending Club es una plataforma de préstamos entre particulares (peer-to-peer) en la que miles de personas solicitan financiamiento para diversos fines. Sin embargo, no todos los solicitantes cumplen con sus obligaciones de pago, lo que representa un riesgo financiero importante para los inversionistas.

Este proyecto tiene como objetivo analizar un conjunto de datos reales de Lending Club y aplicar técnicas de aprendizaje automático para:

- **Predecir** si un préstamo será pagado o no (clasificación supervisada).
- **Agrupar** a los clientes según patrones comunes (clustering no supervisado).

Para lograr esto, se seguirá un flujo estructurado que incluye exploración de datos, limpieza, creación de una muestra representativa, división de conjuntos de entrenamiento y prueba, entrenamiento de modelos, evaluación de resultados y conclusiones.

El enfoque utilizado busca ser reproducible, escalable y fácilmente adaptable a otras plataformas de análisis de crédito.

---


# **1. Inicialización del entorno**

En esta sección se configura el entorno de desarrollo necesario para procesar grandes volúmenes de datos utilizando PySpark.

Además, se importarán todas las librerías necesarias, agrupadas por funcionalidad: transformación de datos, modelado, evaluación y visualización.


## 1.1 Importación de librerías

A continuación se realiza la importación de todas las librerías necesarias para el desarrollo del proyecto.

Estas se encuentran organizadas por funcionalidad:

- **Sesión Spark:** creación y configuración del entorno distribuido.
- **Transformaciones:** funciones para manipulación y limpieza de datos.
- **Preprocesamiento:** herramientas para preparar los datos antes de entrenar modelos.
- **Modelado supervisado:** Random Forest.
- **Modelado no supervisado:** K-means 
- **Evaluación de modelos:** métricas para clasificación binaria y multiclase.
- **Visualización y análisis:** gráficas y análisis complementarios.
- **Herramientas adicionales:** utilidades de `scikit-learn` para validación y métricas.


In [2]:
# -------------------------
# SESIÓN SPARK Y CONFIGURACIÓN
# -------------------------
from pyspark.sql import SparkSession

# -------------------------
# TRANSFORMACIONES EN SPARK
# -------------------------
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col, when, isnan, count, round, regexp_replace, expr, lit,
    percentile_approx, first, sum, skewness, log1p,concat_ws
)
from pyspark.sql.types import IntegerType

# -------------------------
# PREPROCESAMIENTO
# -------------------------
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline

# -------------------------
# MODELOS SUPERVISADOS
# -------------------------
from pyspark.ml.classification import RandomForestClassifier

# -------------------------
# MODELOS NO SUPERVISADOS
# -------------------------
from pyspark.ml.clustering import KMeans

# -------------------------
# EVALUACIÓN DE MODELOS
# -------------------------
from pyspark.ml.evaluation import (
    MulticlassClassificationEvaluator,
    BinaryClassificationEvaluator
)

# -------------------------
# ANÁLISIS Y VISUALIZACIÓN
# -------------------------
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# -------------------------
# LIBRERÍAS ADICIONALES DE SKLEARN
# -------------------------
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold


## 1.2 Creación de la sesión Spark

Apache Spark requiere una sesión activa para poder ejecutar operaciones de análisis y procesamiento distribuido.

En esta sub-sección se inicializa la sesión de Spark con configuraciones personalizadas para uso local. Estas configuraciones están optimizadas para aprovechar todos los núcleos de procesamiento disponibles y asignar suficiente memoria, lo cual es especialmente útil cuando se trabaja con grandes volúmenes de datos como los de Lending Club.

Las configuraciones incluyen:

- Nombre de la aplicación.
- Modo de ejecución local (`local[*]`).
- Asignación de memoria al driver y a los ejecutores.
- Parámetros de paralelismo y particionamiento por defecto.



In [3]:
spark = SparkSession.builder \
    .appName("Proyecto Final") \
    .master("local[*]") \
    .config("spark.driver.memory", "48g") \
    .config("spark.executor.memory", "48g") \
    .config("spark.sql.shuffle.partitions", "32") \
    .config("spark.default.parallelism", "32") \
    .getOrCreate()
spark

# **2. Carga y exploración inicial de los datos**

En esta sección se realiza la importación del conjunto de datos original de Lending Club y una primera exploración para entender su estructura.

Este dataset contiene registros históricos de préstamos otorgados a través de la plataforma Lending Club, incluyendo múltiples variables relacionadas con los solicitantes (como nivel de ingresos, tipo de vivienda o antigüedad laboral) y con los préstamos (como monto solicitado, tasa de interés, estado del préstamo, entre otros).

El objetivo principal de esta sección es:

- Cargar el archivo `loan.csv` con datos reales.
- Visualizar las primeras filas del dataset.
- Analizar la estructura general del conjunto de datos: tipos de variables, número de registros, presencia de valores nulos, etc.
- Obtener estadísticas descriptivas que permitan tener un panorama inicial del comportamiento de los datos.


## 2.1 Carga del dataset

Para este proyecto se utilizará un archivo en formato `.csv` con datos históricos de préstamos proporcionados por la plataforma Lending Club.

Este archivo contiene una gran variedad de variables que describen las características financieras y demográficas de los solicitantes, así como información sobre los préstamos otorgados y su estado final.

En esta sub-sección se realiza la carga del archivo `loan.csv` utilizando PySpark. Para ello se emplean dos configuraciones clave:

- `header=True`: para que Spark reconozca la primera fila como encabezado con nombres de columna.
- `inferSchema=True`: para que Spark detecte automáticamente el tipo de dato de cada columna.

Una vez cargado el archivo, se visualizarán las primeras filas del conjunto de datos y el esquema inferido.



In [4]:
# Cargar el dataset
df = spark.read.option("header", True).option("inferSchema", True).csv("./loan.csv")

# Mostrar las primeras filas
df.show(5)

+----+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------+----------+--------------+----------+-------------------+--------+-----------+----------+----+----+------------------+------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+-------

## 2.2 Vista preliminar del dataset

Después de cargar los datos, es fundamental realizar una exploración inicial para conocer la estructura general del dataset.

En esta sub-sección se revisarán los siguientes aspectos:

- Número total de registros (filas) y columnas.
- Esquema del DataFrame, incluyendo nombre y tipo de dato de cada variable.
- Presencia de valores nulos por columna.
- Estadísticas descriptivas de las variables numéricas (media, desviación estándar, mínimo, máximo, etc.).

Esta información servirá como base para las decisiones que se tomen en las etapas de limpieza, selección de variables y modelado posterior.


In [5]:
#Número total de registros (filas) y columnas.
filas = df.count()
columnas = len(df.columns)
print(f"Registros: {filas} | Columnas: {columnas}")


Registros: 2260668 | Columnas: 145


In [6]:
# Imprimir el esquema del dataframe: nombres de columnas y tipos de datos
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: strin

In [7]:
# Valores nulos por columna
df.select([
    count(when(col(c).isNull() | isnan(c), c)).alias(c)
    for c in df.columns
]).show(truncate=False)


+-------+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+---------+----------+--------------+----------+-------------------+-------+-----------+----------+-------+-------+-------+-----+--------+----------+----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--

In [8]:
df.summary().show()


+-------+----+---------+------------------+------------------+------------------+----------+------------------+-----------------+-------+---------+--------------------+----------+--------------+------------------+-------------------+---------------+-----------+-----------+----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+------------------+------------------+----------------------+----------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+------------------+-----------------+------------------+------------------+--------------------------+---------------------------+------------------+------------------+-----------------

## 2.3 Conversión inicial de tipos de datos

Durante la carga automática del dataset, Spark puede asignar tipos de datos incorrectos a algunas columnas, especialmente cuando existen valores nulos o formatos inconsistentes.

En esta sub-sección se realizará una conversión manual y controlada de los tipos de dato, asegurando que cada variable tenga el formato adecuado para su análisis posterior. Algunos de los casos más comunes a corregir son:

- Columnas numéricas que aparecen como tipo `string`.
- Fechas que se registran como `string` y deben convertirse a `date`.
- Variables categóricas que se mantienen como `string`, pero que serán transformadas más adelante con codificadores.

Este paso permitirá evitar errores en las siguientes fases del pipeline, como el preprocesamiento y el modelado.


In [9]:
tipos_esperados = {
    "loan_amnt": "int",
    "funded_amnt": "int",
    "funded_amnt_inv": "double",
    "term": "string",
    "int_rate": "double",
    "installment": "double",
    "grade": "string",
    "sub_grade": "string",
    "emp_title": "string",
    "emp_length": "string",
    "home_ownership": "string",
    "annual_inc": "double",
    "verification_status": "string",
    "issue_d": "string",
    "loan_status": "string",
    "pymnt_plan": "string",
    "purpose": "string",
    "title": "string",
    "zip_code": "string",
    "addr_state": "string",
    "dti": "double",
    "delinq_2yrs": "int",
    "earliest_cr_line": "int",
    "inq_last_6mths": "int",
    "open_acc": "int",
    "pub_rec": "int",
    "revol_bal": "int",
    "revol_util": "double",
    "total_acc": "int",
    "initial_list_status": "string",
    "out_prncp": "int",
    "out_prncp_inv": "double",
    "total_pymnt": "double",
    "total_pymnt_inv": "double",
    "total_rec_prncp": "double",
    "total_rec_int": "double",
    "total_rec_late_fee": "double",
    "recoveries": "double",
    "collection_recovery_fee": "double",
    "last_pymnt_d": "string",
    "last_pymnt_amnt": "double",
    "last_credit_pull_d": "string",
    "collections_12_mths_ex_med": "int",
    "policy_code": "int",
    "application_type": "string",
    "acc_now_delinq": "int",
    "tot_coll_amt": "int",
    "tot_cur_bal": "int",
    "total_rev_hi_lim": "int",
    "acc_open_past_24mths": "int",
    "avg_cur_bal": "double",
    "bc_open_to_buy": "int",
    "bc_util": "double",
    "chargeoff_within_12_mths": "double",
    "delinq_amnt": "int",
    "mo_sin_old_il_acct": "int",
    "mo_sin_old_rev_tl_op": "int",
    "mo_sin_rcnt_rev_tl_op": "int",
    "mo_sin_rcnt_tl": "int",
    "mort_acc": "int",
    "mths_since_recent_bc": "int",
    "mths_since_recent_inq": "int",
    "num_accts_ever_120_pd": "int",
    "num_actv_bc_tl": "int",
    "num_actv_rev_tl": "int",
    "num_bc_sats": "int",
    "num_bc_tl": "int",
    "num_il_tl": "int",
    "num_op_rev_tl": "int",
    "num_rev_accts": "int",
    "num_rev_tl_bal_gt_0": "int",
    "num_sats": "int",
    "num_tl_120dpd_2m": "int",
    "num_tl_30dpd": "int",
    "num_tl_90g_dpd_24m": "int",
    "num_tl_op_past_12m": "int",
    "pct_tl_nvr_dlq": "double",
    "percent_bc_gt_75": "double",
    "pub_rec_bankruptcies": "int",
    "tax_liens": "int",
    "tot_hi_cred_lim": "int",
    "total_bal_ex_mort": "int",
    "total_bc_limit": "int",
    "total_il_high_credit_limit": "int",
    "hardship_flag": "string",
    "disbursement_method": "string",
    "debt_settlement_flag": "string",
}


In [10]:
# Corrección automática de tipos según tipos_esperados
for columna, tipo_esperado in tipos_esperados.items():
    if columna in dict(df.dtypes):
        tipo_actual = dict(df.dtypes)[columna]
        if tipo_actual != tipo_esperado:
            print(f"Corrigiendo: {columna} de {tipo_actual} a {tipo_esperado}")
            
            # Limpieza de strings si es necesario para convertir a double
            if tipo_esperado == "double" and tipo_actual == "string":
                df = df.withColumn(columna, regexp_replace(col(columna), "[,%$]", "").cast("double"))
            else:
                df = df.withColumn(columna, col(columna).cast(tipo_esperado))


Corrigiendo: annual_inc de string a double
Corrigiendo: dti de string a double
Corrigiendo: delinq_2yrs de string a int
Corrigiendo: earliest_cr_line de string a int
Corrigiendo: inq_last_6mths de string a int
Corrigiendo: open_acc de string a int
Corrigiendo: pub_rec de string a int
Corrigiendo: revol_bal de string a int
Corrigiendo: revol_util de string a double
Corrigiendo: total_acc de string a int
Corrigiendo: out_prncp de string a int
Corrigiendo: out_prncp_inv de string a double
Corrigiendo: total_pymnt de string a double
Corrigiendo: total_pymnt_inv de string a double
Corrigiendo: total_rec_prncp de string a double
Corrigiendo: total_rec_int de string a double
Corrigiendo: total_rec_late_fee de string a double
Corrigiendo: recoveries de string a double
Corrigiendo: collection_recovery_fee de string a double
Corrigiendo: last_pymnt_amnt de string a double
Corrigiendo: collections_12_mths_ex_med de string a int
Corrigiendo: policy_code de string a int
Corrigiendo: acc_now_delinq 

# **3. Limpieza y preprocesamiento**

Una vez cargados y tipificados correctamente los datos, es necesario preparar el conjunto de datos para su análisis y modelado. Esta etapa es crucial para garantizar la calidad de los resultados y evitar errores durante el entrenamiento de los modelos.

Las tareas que se llevarán a cabo en esta sección incluyen:

- Eliminación de columnas irrelevantes, redundantes o con demasiados valores nulos.
- Tratamiento de valores faltantes.
- Transformación de variables categóricas.
- Generación de nuevas variables (en caso necesario).
- Normalización o escalado si se considera útil.
- Preparación final del conjunto de datos con las variables seleccionadas para el modelado.

Estas transformaciones permitirán reducir la complejidad del dataset, mejorar la calidad de los datos y facilitar el aprendizaje de los modelos que se implementarán posteriormente.


## 3.1 Eliminación de columnas con muchos valores nulos

Uno de los primeros pasos fundamentales en la limpieza de datos es identificar y eliminar aquellas columnas que contienen un alto porcentaje de valores nulos.

Las columnas con un exceso de valores faltantes suelen ser poco confiables o difíciles de imputar sin introducir sesgos significativos. En este proyecto se define un umbral del **30%**: todas las columnas que tengan más del 30% de valores nulos serán eliminadas.

Esta decisión busca mantener la calidad de la información sin perder demasiados atributos útiles.


In [11]:
#ELminar columans con el 30% de vlaores nuloso 
# Calcular total de registros
total_rows = df.count()

# Calcular porcentaje de nulos por columna y quedarte con las que tienen menos del 20%
col_porcentaje_nulos = [
   (sum(when((col(c).isNull()) | (col(c) == ''), 1).otherwise(0)) / total_rows).alias(c)
    for c in df.columns
]

# Generar DataFrame con porcentaje de nulos
porcentaje_nulos_df = df.select(col_porcentaje_nulos)
porcentaje_nulos = porcentaje_nulos_df.collect()[0].asDict()

# Filtrar columnas que tienen menos de 20% de nulos
columnas_validas = [k for k, v in porcentaje_nulos.items() if v < 0.3]

# Seleccionar solo las columnas válidas
df = df.select(*columnas_validas)

print(f"Columnas seleccionadas con <30% de nulos: {len(columnas_validas)}")


Columnas seleccionadas con <30% de nulos: 86


## 3.2 Eliminación de columnas categóricas con alta cardinalidad

Las columnas categóricas con demasiados valores únicos (alta cardinalidad) pueden ser problemáticas para los modelos de aprendizaje automático, especialmente si no aportan información útil o presentan una alta dispersión.

En este proyecto, se define un umbral de **10,000 valores distintos**: todas las columnas categóricas que excedan este número serán eliminadas del conjunto de datos.

Este umbral se basa en dos criterios:

- Dificultad de codificación eficiente (OneHotEncoding o StringIndexer).
- Posible irrelevancia de columnas con demasiadas categorías específicas (como títulos de empleo, nombres personalizados, etc.).

La eliminación de estas columnas permite reducir la complejidad del dataset y mejorar el rendimiento del pipeline de procesamiento y modelado.


In [12]:
# Identificar columnas de tipo string con más de 10,000 valores únicos
columnas_texto_libre = []

for c in df.columns:
    if dict(df.dtypes)[c] == "string":
        num_valores_unicos = df.select(c).distinct().count()
        if num_valores_unicos > 10000:
            columnas_texto_libre.append(c)

# Mostrar las columnas identificadas
print("Columnas con alta cardinalidad (>10,000 categorías únicas):")
print(columnas_texto_libre)

# Eliminar las columnas del DataFrame
df = df.drop(*columnas_texto_libre)

print(f"Número de columnas eliminadas: {len(columnas_texto_libre)}")
print(f"Número de columnas restantes: {len(df.columns)}")



Columnas con alta cardinalidad (>10,000 categorías únicas):
['emp_title', 'title']
Número de columnas eliminadas: 2
Número de columnas restantes: 84


## 3.3 Eliminación de columnas numéricas altamente correlacionadas

Las variables numéricas altamente correlacionadas entre sí pueden introducir redundancia en el modelo y dificultar la interpretación de los resultados. Además, en algunos algoritmos como regresión logística, estas correlaciones pueden provocar problemas de multicolinealidad.

Para mitigar esto, en esta sub-sección se calculará la **matriz de correlación de Pearson** entre todas las variables numéricas y se eliminarán aquellas columnas cuya correlación absoluta con otra columna sea **mayor a 0.95**.

Este umbral busca conservar únicamente una de las dos variables fuertemente correlacionadas, reduciendo la dimensionalidad del conjunto de datos sin pérdida significativa de información.


In [13]:
# Seleccionar columnas numéricas del DataFrame
columnas_numericas = [c for c, t in df.dtypes if t in ("double", "int")]

# Tomar una muestra del 10% del dataset para evitar saturar memoria
df_sample_pd = df.select(columnas_numericas).sample(False, 0.10, seed=1).toPandas()

# Calcular matriz de correlación absoluta de Pearson
cor_matrix = df_sample_pd.corr().abs()

# Umbral para eliminar columnas altamente correlacionadas
umbral = 0.95
columnas_correladas = set()

# Recorrer matriz de correlación y marcar columnas redundantes
for i in range(len(cor_matrix.columns)):
    for j in range(i + 1, len(cor_matrix.columns)):
        col1 = cor_matrix.columns[i]
        col2 = cor_matrix.columns[j]
        if cor_matrix.iloc[i, j] > umbral:
            print(f"Alta correlación entre: {col1} y {col2} (r = {cor_matrix.iloc[i, j]:.4f})")
            columnas_correladas.add(col2)  # Conservamos col1, eliminamos col2

# Eliminar columnas redundantes del DataFrame original
df = df.drop(*list(columnas_correladas))

# Mostrar resultados
print(f"Columnas eliminadas por redundancia lineal: {len(columnas_correladas)}")
print(f"Total de columnas restantes: {len(df.columns)}")


Alta correlación entre: loan_amnt y funded_amnt (r = 0.9998)
Alta correlación entre: loan_amnt y funded_amnt_inv (r = 0.9990)
Alta correlación entre: funded_amnt y funded_amnt_inv (r = 0.9993)
Alta correlación entre: open_acc y num_sats (r = 0.9990)
Alta correlación entre: out_prncp y out_prncp_inv (r = 0.9999)
Alta correlación entre: total_pymnt y total_pymnt_inv (r = 0.9992)
Alta correlación entre: total_pymnt y total_rec_prncp (r = 0.9671)
Alta correlación entre: total_pymnt_inv y total_rec_prncp (r = 0.9665)
Alta correlación entre: tot_cur_bal y tot_hi_cred_lim (r = 0.9793)
Alta correlación entre: num_actv_rev_tl y num_rev_tl_bal_gt_0 (r = 0.9835)
Columnas eliminadas por redundancia lineal: 8
Total de columnas restantes: 76


## 3.4 Eliminación de columnas con baja varianza

Las columnas con muy poca variación en sus valores no aportan información significativa a los modelos de aprendizaje automático, ya que no ayudan a diferenciar entre clases ni a establecer patrones útiles.

En esta sub-sección se eliminarán:

- Columnas numéricas cuya varianza sea igual a 0.
- Columnas categóricas con una única categoría (baja varianza cualitativa).

Estas columnas no solo son irrelevantes para el aprendizaje del modelo, sino que también pueden aumentar el tiempo de procesamiento y generar ruido.

Este filtrado busca conservar únicamente aquellas variables que ofrecen información útil para la clasificación o segmentación.


In [14]:
# Umbral para considerar que una variable tiene baja varianza (70%)
umbral = 0.70
total_rows = df.count()

# ---------------------------
# Categóricas con baja varianza
# ---------------------------
cat_columns = [c for c, t in df.dtypes if t == "string"]
columnas_baja_varianza = []

for c in cat_columns:
    top_valor = df.groupBy(c).count().orderBy("count", ascending=False).first()
    if top_valor and (top_valor["count"] / total_rows) >= umbral:
        print(f"{c}: {top_valor['count'] / total_rows:.2%} del dataset tiene '{top_valor[c]}'")
        columnas_baja_varianza.append(c)

# Eliminar columnas categóricas con baja varianza
df = df.drop(*columnas_baja_varianza)
print(f"\nColumnas categóricas eliminadas por baja varianza: {len(columnas_baja_varianza)}")


# ---------------------------
# Numéricas con valor dominante (>70%)
# ---------------------------
numeric_columns = [c for c, t in df.dtypes if t in ("double", "int")]
columnas_valor_dominante = []

for c in numeric_columns:
    valor_mas_frecuente = (
        df.groupBy(c)
        .count()
        .orderBy("count", ascending=False)
        .first()
    )

    if valor_mas_frecuente:
        valor = valor_mas_frecuente[0]
        cuenta = valor_mas_frecuente["count"]
        porcentaje = cuenta / total_rows

        if porcentaje >= umbral:
            print(f"{c}: {porcentaje:.2%} de los valores son '{valor}'")
            columnas_valor_dominante.append(c)

# Eliminar columnas numéricas con valor dominante
df = df.drop(*columnas_valor_dominante)
print(f"\nColumnas numéricas eliminadas por valor dominante: {len(columnas_valor_dominante)}")
print(f"Total de columnas restantes: {len(df.columns)}")

term: 71.21% del dataset tiene ' 36 months'
pymnt_plan: 99.97% del dataset tiene 'n'
application_type: 94.65% del dataset tiene 'Individual'
hardship_flag: 99.95% del dataset tiene 'N'
disbursement_method: 96.53% del dataset tiene 'Cash'
debt_settlement_flag: 98.53% del dataset tiene 'N'

Columnas categóricas eliminadas por baja varianza: 6
delinq_2yrs: 81.34% de los valores son '0'
pub_rec: 84.16% de los valores son '0'
total_rec_late_fee: 96.25% de los valores son '0.0'
recoveries: 92.14% de los valores son '0.0'
collection_recovery_fee: 92.52% de los valores son '0.0'
collections_12_mths_ex_med: 98.33% de los valores son '0'
policy_code: 99.99% de los valores son '1'
acc_now_delinq: 99.60% de los valores son '0'
tot_coll_amt: 82.11% de los valores son '0'
chargeoff_within_12_mths: 99.22% de los valores son '0.0'
delinq_amnt: 99.67% de los valores son '0'
num_accts_ever_120_pd: 74.64% de los valores son '0'
num_tl_120dpd_2m: 93.15% de los valores son '0'
num_tl_30dpd: 96.63% de los v

## 3.5 Eliminación de columnas con fuga de información

La fuga de información (data leakage) ocurre cuando se incluyen en el entrenamiento variables que contienen directa o indirectamente información sobre el resultado que se pretende predecir. Esto genera modelos artificialmente precisos en fase de entrenamiento, pero poco útiles en escenarios reales.

En este proyecto se eliminarán todas las columnas que:

- Contengan información que solo está disponible después del préstamo (como pagos realizados, saldos finales, o fechas de pago).
- Estén altamente relacionadas con la variable objetivo (`loan_status`) por diseño.

Este paso es crucial para asegurar que el modelo aprenda únicamente a partir de información disponible en el momento de la solicitud del préstamo, imitando una situación real.


In [15]:
#Columnas eliminadas por contener información posterior al préstamo
columnas_fuga_info = [
    "issue_d",             # Fecha del préstamo
    "last_pymnt_d",        # Fecha del último pago
    "last_credit_pull_d",  # Última revisión de crédito
]

df = df.drop(*[c for c in columnas_fuga_info if c in df.columns])
print(f"Columnas eliminadas por fuga de información: {len(columnas_fuga_info)}")
print(f"Total de columnas restantes en el DataFrame: {len(df.columns)}")

Columnas eliminadas por fuga de información: 3
Total de columnas restantes en el DataFrame: 50


## 3.6 Eliminación de columnas irrelevantes o redundantes

Además de eliminar columnas con fuga de información, es importante revisar y eliminar variables que no aportan valor al análisis por otras razones, como:

- **Irrelevancia:** columnas con información personal, metadatos técnicos o atributos poco explicativos.
- **Redundancia:** variables que están representadas de forma más general o repetida en otras columnas.
- **Diseño técnico:** variables calculadas automáticamente que no reflejan comportamientos reales ni tienen utilidad predictiva.

La eliminación de estas columnas permite simplificar el conjunto de datos, reducir la dimensionalidad y evitar sesgos innecesarios en los modelos.


In [16]:
# Columnas consideradas irrelevantes por diseño o redundancia obvia
columnas_irrelevantes = [
    "zip_code",           # Alta cardinalidad, dato personal
    "addr_state",         # Baja capacidad explicativa
    "sub_grade",          # Ya contenida en 'grade'
    "initial_list_status" # Metadata técnica
]

df = df.drop(*[c for c in columnas_irrelevantes if c in df.columns])
print(f"Columnas eliminadas por irrelevancia justificada: {len(columnas_irrelevantes)}")

# Columnas redundantes por definición técnica
columnas_a_eliminar_por_definicion = [
    "bc_open_to_buy",
    "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op",
    "mo_sin_rcnt_rev_tl_op",
    "mo_sin_rcnt_tl",
    "num_bc_tl",
    "num_op_rev_tl",
    "num_rev_accts",
    "total_bc_limit"
]

df = df.drop(*[c for c in columnas_a_eliminar_por_definicion if c in df.columns])
print(f"Columnas eliminadas por redundancia técnica: {len(columnas_a_eliminar_por_definicion)}")
print(f"Total de columnas restantes en el DataFrame: {len(df.columns)}")

Columnas eliminadas por irrelevancia justificada: 4
Columnas eliminadas por redundancia técnica: 9
Total de columnas restantes en el DataFrame: 37


### 3.7 Imputación de valores faltantes

Después de eliminar columnas con un alto porcentaje de valores nulos, aún pueden existir columnas que contienen algunos valores faltantes. Para asegurar la integridad del dataset y evitar errores en las etapas siguientes del pipeline, se procederá a imputar estos valores.

Se utilizarán las siguientes estrategias:

- **Para variables numéricas:** se imputará utilizando la **media** o la **mediana**, según la distribución de la variable.
- **Para variables categóricas:** se imputará con la **moda** (valor más frecuente).

Estas técnicas son simples pero efectivas, y permiten mantener una aproximación conservadora al comportamiento de cada variable.


In [17]:
# Mostrar número de valores nulos por columna
print("Valores nulos por columna:")
df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show(truncate=False)


Valores nulos por columna:
+---------+--------+-----------+-----+----------+--------------+----------+-------------------+-----------+-------+----+--------------+--------+---------+----------+---------+---------+-----------+-------------+---------------+-----------+----------------+--------------------+-----------+-------+--------+--------------------+---------------------+--------------+---------------+-----------+---------+------------------+--------------+----------------+-----------------+--------------------------+
|loan_amnt|int_rate|installment|grade|emp_length|home_ownership|annual_inc|verification_status|loan_status|purpose|dti |inq_last_6mths|open_acc|revol_bal|revol_util|total_acc|out_prncp|total_pymnt|total_rec_int|last_pymnt_amnt|tot_cur_bal|total_rev_hi_lim|acc_open_past_24mths|avg_cur_bal|bc_util|mort_acc|mths_since_recent_bc|mths_since_recent_inq|num_actv_bc_tl|num_actv_rev_tl|num_bc_sats|num_il_tl|num_tl_op_past_12m|pct_tl_nvr_dlq|percent_bc_gt_75|total_bal_ex_mort|tot

In [18]:
numeric_columns = [c for c, t in df.dtypes if t in ("double", "int")]
# Imputación de variables numéricas con MEDIANA
for col_name in numeric_columns:
    mediana = df.approxQuantile(col_name, [0.5], 0.01)[0]
    df = df.withColumn(
        col_name,
        when(col(col_name).isNull(), lit(mediana)).otherwise(col(col_name))
    )
cat_columns = [c for c, t in df.dtypes if t == "string"]
# Imputación de variables categóricas con MODO (valor más frecuente)
for col_name in cat_columns:
    modo = (df.groupBy(col_name)
                     .count()
                     .orderBy(F.desc("count"))
                     .first()[0])
    df = df.withColumn(
        col_name,
        when(col(col_name).isNull(), lit(modo)).otherwise(col(col_name))
    )


In [19]:
# Mostrar número de valores nulos por columna
print("Valores nulos por columna:")
df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show(truncate=False)

Valores nulos por columna:
+---------+--------+-----------+-----+----------+--------------+----------+-------------------+-----------+-------+---+--------------+--------+---------+----------+---------+---------+-----------+-------------+---------------+-----------+----------------+--------------------+-----------+-------+--------+--------------------+---------------------+--------------+---------------+-----------+---------+------------------+--------------+----------------+-----------------+--------------------------+
|loan_amnt|int_rate|installment|grade|emp_length|home_ownership|annual_inc|verification_status|loan_status|purpose|dti|inq_last_6mths|open_acc|revol_bal|revol_util|total_acc|out_prncp|total_pymnt|total_rec_int|last_pymnt_amnt|tot_cur_bal|total_rev_hi_lim|acc_open_past_24mths|avg_cur_bal|bc_util|mort_acc|mths_since_recent_bc|mths_since_recent_inq|num_actv_bc_tl|num_actv_rev_tl|num_bc_sats|num_il_tl|num_tl_op_past_12m|pct_tl_nvr_dlq|percent_bc_gt_75|total_bal_ex_mort|total

## 3.8 Revisión de sesgos, outliers y transformaciones

Una vez imputados los valores faltantes, es importante revisar la presencia de valores atípicos (**outliers**) y distribuciones sesgadas (**skewed distributions**) en las variables numéricas.

En esta sub-sección se realiza un análisis combinado de:

- **Asimetría (skewness):** para identificar variables con distribuciones fuertemente sesgadas (asimetría mayor a ±1).
- **Outliers:** mediante el rango intercuartílico (IQR), para detectar variables con una cantidad significativa de valores extremos.

Con base en este análisis, se toma una decisión automática para cada variable numérica:

- **Aplicar transformación logarítmica** (`log1p`) en variables con alta asimetría positiva y más de 10,000 outliers.
- **Eliminar outliers** si la variable tiene valores extremos pero no presenta alta asimetría.
- **Conservar la variable tal como está** si no hay problemas detectados.

Estas transformaciones ayudan a estabilizar la varianza, mejorar la distribución de los datos y aumentar el desempeño de los modelos de aprendizaje.


In [20]:
def eliminar_outliers_spark(df, columnas):
    for nombre_col in columnas:
        # Calcular Q1 y Q3
        q1, q3 = df.approxQuantile(nombre_col, [0.25, 0.75], 0.01)
        iqr = q3 - q1
        limite_inferior = q1 - 1.5 * iqr
        limite_superior = q3 + 1.5 * iqr

        # Filtrar fuera del IQR
        df = df.filter(
            (col(nombre_col) >= limite_inferior) & (col(nombre_col) <= limite_superior)
        )
        
    return df

# 1. Calcular skewness (asimetría) para cada variable numérica
skew_data = df.select([skewness(c).alias(c) for c in numeric_columns]).toPandas().T
skew_data.columns = ['skewness']
skew_data['feature'] = skew_data.index

# 2. Identificar outliers usando el rango intercuartílico (IQR)
outlier_info = []

for col_name in numeric_columns:
    q1, q3 = df.approxQuantile(col_name, [0.25, 0.75], 0.01)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    outlier_count = df.filter((col(col_name) < lower) | (col(col_name) > upper)).count()
    outlier_info.append((col_name, outlier_count))

outlier_df = pd.DataFrame(outlier_info, columns=["feature", "outlier_count"])

# 3. Fusionar skewness + outlier info
merged = skew_data.merge(outlier_df, on="feature")

# 4. Clasificar columnas en 3 categorías: aplicar log, eliminar outliers o conservar
merged["decision"] = merged.apply(lambda row:
    "log" if abs(row["skewness"]) > 1.0 and row["outlier_count"] > 10000 else
    "delete" if row["outlier_count"] > 0 and abs(row["skewness"]) < 1.0 else
    "keep", axis=1
)

# 5. Extraer listas por decisión
cols_log = merged[merged["decision"] == "log"]["feature"].tolist()
cols_delete = merged[merged["decision"] == "delete"]["feature"].tolist()
cols_keep = merged[merged["decision"] == "keep"]["feature"].tolist()

# 6. Mostrar resumen de decisiones
print("Columnas para log:", cols_log)
print("Columnas para eliminar outliers:", cols_delete)
print("Columnas para conservar:", cols_keep)

# elimina outliers según IQR.
df_v1 = eliminar_outliers_spark(df, cols_delete)

# Aplicar transformación logarítmica a columnas con alta asimetría y muchos outliers
for c in cols_log:
    df_v1 = df_v1.withColumn(c, log1p(col(c)))

print("Transformaciones aplicadas: eliminación de outliers y log.")


Columnas para log: ['installment', 'annual_inc', 'dti', 'inq_last_6mths', 'open_acc', 'revol_bal', 'total_acc', 'out_prncp', 'total_pymnt', 'total_rec_int', 'last_pymnt_amnt', 'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths', 'avg_cur_bal', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_il_tl', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'total_bal_ex_mort', 'total_il_high_credit_limit']
Columnas para eliminar outliers: ['loan_amnt', 'int_rate', 'bc_util']
Columnas para conservar: ['revol_util', 'percent_bc_gt_75']
Transformaciones aplicadas: eliminación de outliers y log.


## 3.9 Observación final del dataset limpio

Antes de pasar a la etapa de preprocesamiento para modelado, es importante validar el estado actual del conjunto de datos tras todas las tareas de limpieza realizadas.

En esta sub-sección se observará:

- Número total de filas y columnas restantes.
- Número de columnas por tipo de dato.
- Presencia residual de valores nulos (si los hubiera).
- Distribución general de algunas variables clave.
- Estadísticas descriptivas para variables numéricas
- Estadísticas descriptivas para variables categoricas 
Este resumen final sirve como verificación de que el dataset se encuentra en condiciones óptimas para pasar a las fases de transformación, codificación y modelado.


In [21]:
# -----------------------------
# 1. Número total de filas y columnas
# -----------------------------
num_filas = df_v1.count()
num_columnas = len(df_v1.columns)
print(f"Total de registros: {num_filas}")
print(f"Total de columnas: {num_columnas}")

# -----------------------------
# 2. Conteo de columnas por tipo de dato
# -----------------------------
tipos = {}
for col_name, tipo in df_v1.dtypes:
    tipos[tipo] = tipos.get(tipo, 0) + 1

print("\nConteo de columnas por tipo de dato:")
for tipo, cantidad in tipos.items():
    print(f" - {tipo}: {cantidad}")

# -----------------------------
# 3. Verificación de valores nulos residuales
# -----------------------------
print("\nValores nulos restantes por columna:")
df_v1.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_v1.columns
]).show(truncate=False)

# Eliminar registros que aún contengan valores nulos
filas_antes = df_v1.count()
df_v1 = df_v1.na.drop()
filas_despues = df_v1.count()

print(f"Registros eliminados por contener valores nulos: {filas_antes - filas_despues}")
print(f"Total de registros restantes: {filas_despues}")

# -----------------------------
# 4. Vista de muestra final
# -----------------------------
print("\n Vista de muestra del dataset final:")
df_v1.show(5)


Total de registros: 2178236
Total de columnas: 37

Conteo de columnas por tipo de dato:
 - double: 31
 - string: 6

Valores nulos restantes por columna:
+---------+--------+-----------+-----+----------+--------------+----------+-------------------+-----------+-------+---+--------------+--------+---------+----------+---------+---------+-----------+-------------+---------------+-----------+----------------+--------------------+-----------+-------+--------+--------------------+---------------------+--------------+---------------+-----------+---------+------------------+--------------+----------------+-----------------+--------------------------+
|loan_amnt|int_rate|installment|grade|emp_length|home_ownership|annual_inc|verification_status|loan_status|purpose|dti|inq_last_6mths|open_acc|revol_bal|revol_util|total_acc|out_prncp|total_pymnt|total_rec_int|last_pymnt_amnt|tot_cur_bal|total_rev_hi_lim|acc_open_past_24mths|avg_cur_bal|bc_util|mort_acc|mths_since_recent_bc|mths_since_recent_inq|n

In [22]:
#Estadísticas descriptivas para variables numéricas
numeric_columns = [col.name for col in df.schema.fields if col.dataType.simpleString() != 'string']
df_numerics = df.select(numeric_columns)
df_numerics.describe().show()

+-------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+--------------------+------------------+-----------------+------------------+--------------------+---------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+-----------------+--------------------------+
|summary|         loan_amnt|          int_rate|      installment|        annual_inc|               dti|    inq_last_6mths|          open_acc|        revol_bal|        revol_util|         total_acc|        out_prncp|       total_pymnt|     total_rec_int|  last_pymnt_amnt|      tot_cur_bal| total_rev_hi_lim|acc_open_past_24mths|       avg_cur_bal|          bc_util|          mort_acc|mths_since_re

In [23]:
#Estadísticas descriptivas para variables categoricas 
cat_columns = [col.name for col in df.schema.fields if col.dataType.simpleString() == 'string']
df_strings = df.select(cat_columns)
for col_name in cat_columns:
    total_distintos = df.select(col_name).distinct().count()
    df_strings.groupBy(col_name).count().orderBy("count", ascending=False).show()

+-----+------+
|grade| count|
+-----+------+
|    B|663557|
|    C|650053|
|    A|433027|
|    D|324424|
|    E|135639|
|    F| 41800|
|    G| 12168|
+-----+------+

+----------+------+
|emp_length| count|
+----------+------+
| 10+ years|748005|
|   2 years|203676|
|  < 1 year|189988|
|   3 years|180753|
|    1 year|148403|
|       n/a|146907|
|   5 years|139698|
|   4 years|136605|
|   6 years|102628|
|   7 years| 92695|
|   8 years| 91914|
|   9 years| 79395|
| reactors"|     1|
+----------+------+

+--------------+-------+
|home_ownership|  count|
+--------------+-------+
|      MORTGAGE|1111449|
|          RENT| 894929|
|           OWN| 253057|
|           ANY|    996|
|         OTHER|    182|
|          NONE|     54|
|       2 years|      1|
+--------------+-------+

+-------------------+------+
|verification_status| count|
+-------------------+------+
|    Source Verified|886230|
|       Not Verified|744806|
|           Verified|629631|
|              38000|     1|
+-------------

# **4. Construcción de la muestra representativa (M)**

En esta sección se construirá una muestra representativa (`M`) a partir del conjunto original de datos, con el objetivo de facilitar el entrenamiento de modelos sin perder la diversidad ni las proporciones clave de la población.

Para ello, se realizarán las siguientes etapas:

1. **Agrupación de categorías poco frecuentes** en las variables `home_ownership` y `purpose`, con el fin de reducir la complejidad del análisis y evitar combinaciones con muy baja representatividad.

2. **Identificación de combinaciones significativas** entre las variables `purpose`, `home_ownership` y `verification_status`, seleccionando únicamente aquellas con más de 50,000 registros para garantizar solidez estadística.

3. **Aplicación de muestreo aleatorio** del 20% sobre cada combinación válida, generando subconjuntos balanceados (`Mi`) que capturan la estructura poblacional de forma proporcional.

4. **Unión de los subconjuntos `Mi`** para conformar la muestra final `M`, la cual será utilizada en las siguientes etapas del proceso de modelado.

Este procedimiento permite reducir el volumen total de datos manteniendo la calidad y representatividad necesarias para entrenar modelos de aprendizaje automático de manera eficiente.


## 4.1 Agrupación de categorías relevantes

Para reducir la dimensionalidad de las variables categóricas y facilitar una mejor caracterización de la población, se agruparon algunas categorías de baja frecuencia en nuevas clases.

- En `home_ownership`, se agruparon todas las categorías poco frecuentes en una nueva categoría llamada `OTHER`.
- En `purpose`, se unificaron propósitos similares en grupos temáticos: `debt_related`, `large_expense`, `personal_use`, `life_event` y `other`.


In [28]:
# Agrupar categorías frecuentes en 'home_ownership'
categorias_frecuentes = ['MORTGAGE', 'RENT', 'OWN']
df_v1 = df_v1.withColumn(
    "home_ownership",
    when(col("home_ownership").isin(categorias_frecuentes), col("home_ownership"))
    .otherwise("OTHER")
)

# Agrupar categorías en 'purpose'
df_v1 = df_v1.withColumn(
    "purpose",
    when(col("purpose").isin("debt_consolidation", "credit_card"), "debt_related")
    .when(col("purpose").isin("home_improvement", "major_purchase"), "large_expense")
    .when(col("purpose").isin("small_business", "car", "vacation", "house"), "personal_use")
    .when(col("purpose").isin("medical", "wedding", "moving"), "life_event")
    .otherwise("other")
)

## 4.2 Identificación de combinaciones válidas

Se generaron todas las combinaciones posibles entre las variables `purpose`, `home_ownership` y `verification_status`. Posteriormente, se calcularon sus frecuencias relativas y absolutas.

Solo se consideraron aquellas combinaciones con **más de 50,000 registros** para garantizar una muestra robusta y representativa.


In [29]:
total = df_v1.count()

combinaciones = df_v1.groupBy('purpose', 'home_ownership','verification_status') \
    .count() \
    .withColumnRenamed("count", "frecuencia") \
    .withColumn("probabilidad (%)", round((col("frecuencia") / total) * 100, 2)) \
    .orderBy("frecuencia", ascending=False)

print("Combinaciones posibles:", combinaciones.count())
combinaciones.show(truncate=False)

Combinaciones posibles: 13
+-------+--------------+-------------------+----------+----------------+
|purpose|home_ownership|verification_status|frecuencia|probabilidad (%)|
+-------+--------------+-------------------+----------+----------------+
|other  |MORTGAGE      |Source Verified    |397575    |18.25           |
|other  |MORTGAGE      |Not Verified       |362945    |16.66           |
|other  |RENT          |Source Verified    |359338    |16.5            |
|other  |MORTGAGE      |Verified           |306153    |14.06           |
|other  |RENT          |Not Verified       |285314    |13.1            |
|other  |RENT          |Verified           |221683    |10.18           |
|other  |OWN           |Source Verified    |97694     |4.49            |
|other  |OWN           |Not Verified       |83284     |3.82            |
|other  |OWN           |Verified           |63065     |2.9             |
|other  |OTHER         |Not Verified       |485       |0.02            |
|other  |OTHER         |

## 4.3 Muestreo aleatorio de subconjuntos `Mi`

A cada combinación válida se le aplicó un muestreo aleatorio del 20% sin reemplazo (`fraction=0.2`), obteniendo un subconjunto `Mi` por cada combinación.

Estos subconjuntos fueron almacenados en una lista para su posterior consolidación.

In [30]:
# Definir umbral mínimo de registros para considerar una combinación válida
UMBRAL = 50000

# Filtrar las combinaciones que superan el umbral
combinaciones_validas = df_v1.groupBy("purpose", "home_ownership", "verification_status") \
    .count() \
    .filter(col("count") > UMBRAL) \
    .collect()

# Crear lista para almacenar cada subconjunto Mi
muestras = []

# Aplicar muestreo del 20% a cada combinación válida y agregarla a la lista
for fila in combinaciones_validas:
    p, h, v = fila["purpose"], fila["home_ownership"], fila["verification_status"]
    
    mi = df_v1.filter(
        (col("purpose") == p) &
        (col("home_ownership") == h) &
        (col("verification_status") == v)
    )
    
    mi = mi.sample(withReplacement=False, fraction=0.2, seed=1)
    muestras.append(mi)

# Confirmar cuántos subconjuntos se generaron
print("Número de subconjuntos Mi generados:", len(muestras))


Número de subconjuntos Mi generados: 9


### 4.4 Unión de subconjuntos y validación de la muestra `M`

La muestra representativa `M` se construyó como la unión de todos los subconjuntos `Mi`.

Finalmente, se validó que la distribución de combinaciones en `M` reflejara las proporciones generales de la población original.

In [31]:
# Unión final para obtener M
M = muestras[0]
for mi in muestras[1:]:
    M = M.union(mi)

print("Total de registros en la muestra M:", M.count())

# Verifica distribución
M.groupBy("purpose", "home_ownership", "verification_status").count().orderBy("count", ascending=False).show(truncate=False)

Total de registros en la muestra M: 435572
+-------+--------------+-------------------+-----+
|purpose|home_ownership|verification_status|count|
+-------+--------------+-------------------+-----+
|other  |MORTGAGE      |Source Verified    |79418|
|other  |MORTGAGE      |Not Verified       |72520|
|other  |RENT          |Source Verified    |71976|
|other  |MORTGAGE      |Verified           |61301|
|other  |RENT          |Not Verified       |57157|
|other  |RENT          |Verified           |44330|
|other  |OWN           |Source Verified    |19551|
|other  |OWN           |Not Verified       |16713|
|other  |OWN           |Verified           |12606|
+-------+--------------+-------------------+-----+



# **5. Preparación del Dataset para Modelado**

Para asegurar una correcta preparación del dataset sin fugas de información, se siguió este flujo:

1. **Transformación de la variable objetivo**: se creó una columna binaria `label` a partir de `loan_status`, donde 1 indica riesgo y 0 no riesgo.

2. **División del dataset**: se separó la muestra representativa `M` en conjuntos `train` (80%) y `test` (20%) mediante una partición estratificada.

3. **Definición del pipeline**: se construyó un pipeline con `StringIndexer` para codificar variables categóricas y `VectorAssembler` para generar el vector `features`.

4. **Entrenamiento del pipeline**: el pipeline se entrenó únicamente con `train` y luego se aplicó a `test`.

Con esto, ambos conjuntos quedaron listos para el entrenamiento de modelos supervisados y no supervisados.


## 5.1 Transformación de la variable objetivo (`loan_status` → `label`)

La variable `loan_status` indica el estado final del préstamo otorgado. Esta columna contiene múltiples categorías textuales como:

- *Fully Paid* (Pagado completamente)
- *Charged Off* (Incobrable)
- *Default* (Incumplimiento)
- *Late* (Atrasado)

Para el entrenamiento de un modelo de clasificación binaria, esta variable será transformada en una columna numérica llamada **`label`**, con los siguientes valores:

- `0` → Préstamo sin riesgo (ej. *Fully Paid*)
- `1` → Préstamo con riesgo (ej. *Charged Off*, *Default*, *Late*)

Esta conversión permitirá usarla como variable objetivo (`label`) en algoritmos de clasificación supervisada.


In [32]:
# Definir los valores considerados como préstamos con riesgo
valores_riesgo = ["Charged Off", "Default", "Late (31-120 days)", "Late (16-30 days)"]

# Crear una nueva columna binaria 'label':
# - 1 si el préstamo representa riesgo
# - 0 si fue pagado completamente u otro estado sin riesgo
M = M.withColumn(
    "label",
    when(col("loan_status").isin(valores_riesgo), 1).otherwise(0)
)

# Verificar la distribución de clases en la variable objetivo
M.groupBy("label").count().orderBy("label").show()


+-----+------+
|label| count|
+-----+------+
|    0|381284|
|    1| 54288|
+-----+------+



## 5.2 Codificación de variables categóricas

Los algoritmos de aprendizaje automático requieren que todas las variables de entrada sean numéricas. Sin embargo, el dataset incluye varias variables categóricas relevantes que están representadas como cadenas de texto (`string`), como:

- `home_ownership`
- `verification_status`
- `purpose`
- `grade`
- `emp_length`

Para convertirlas en un formato adecuado, se utilizarán las siguientes herramientas de PySpark:

- **`StringIndexer`**: asigna un valor numérico único a cada categoría, preservando una relación de orden si existe.

Estas transformaciones permitirán generar una representación numérica adecuada de las variables categóricas, que será utilizada posteriormente en el ensamblado del vector de características.


In [33]:
# Definir las variables categóricas que serán transformadas
variables_categoricas = ["home_ownership", "verification_status", "purpose", "grade", "emp_length"]

# Crear una lista de transformaciones StringIndexer para cada variable
# 'handleInvalid="keep"' permite manejar valores nuevos o nulos sin error
indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_idx", handleInvalid="keep")
    for col in variables_categoricas
]

# Crear e implementar un pipeline con todos los indexadores
pipeline = Pipeline(stages=indexers)
M = pipeline.fit(M).transform(M)

# Mostrar una vista rápida de las nuevas columnas indexadas
print("Columnas transformadas:")
for c in variables_categoricas:
    print(f"→ {c} → {c}_idx")
    M.select(c, f"{c}_idx").show(5, truncate=False)

Columnas transformadas:
→ home_ownership → home_ownership_idx
+--------------+------------------+
|home_ownership|home_ownership_idx|
+--------------+------------------+
|MORTGAGE      |0.0               |
|MORTGAGE      |0.0               |
|MORTGAGE      |0.0               |
|MORTGAGE      |0.0               |
|MORTGAGE      |0.0               |
+--------------+------------------+
only showing top 5 rows

→ verification_status → verification_status_idx
+-------------------+-----------------------+
|verification_status|verification_status_idx|
+-------------------+-----------------------+
|Verified           |2.0                    |
|Verified           |2.0                    |
|Verified           |2.0                    |
|Verified           |2.0                    |
|Verified           |2.0                    |
+-------------------+-----------------------+
only showing top 5 rows

→ purpose → purpose_idx
+-------+-----------+
|purpose|purpose_idx|
+-------+-----------+
|other  |0.0

## 5.3 Ensamblado del vector de características

Una vez que las variables categóricas han sido codificadas y las variables numéricas han sido depuradas, el siguiente paso consiste en unificar todas las columnas relevantes en una sola estructura que pueda ser interpretada por los modelos de aprendizaje automático de PySpark.

Para ello, se emplea el transformador `VectorAssembler`, el cual toma múltiples columnas numéricas como entrada y las combina en una nueva columna de tipo vector, denominada `features`.

Las columnas seleccionadas para este ensamblado incluyen:

- Las versiones codificadas de las variables categóricas (columnas terminadas en `_idx`).
- Las variables numéricas limpias que serán utilizadas como predictores.

Este paso es indispensable, ya que los modelos en PySpark requieren que las variables de entrada estén agrupadas en una única columna de tipo vector para poder entrenar de forma eficiente.



In [34]:
# Variables categóricas ya codificadas
columnas_categoricas_idx = [f"{col}_idx" for col in variables_categoricas]

# Identificamos automáticamente las numéricas
columnas_utilizadas = columnas_categoricas_idx + [
    c for c in M.columns
    if c not in columnas_categoricas_idx + variables_categoricas + ["loan_status", "label", "features"]
    and M.schema[c].dataType.simpleString() in ['double', 'int']
]

# Ensamblador
ensamblador = VectorAssembler(
    inputCols=columnas_utilizadas,
    outputCol="features"
)

# Aplicar ensamblado
M = ensamblador.transform(M)

# Verificar
print("Ensamblaje exitoso. Vista previa del vector de entrada:")
M.select("features", "label").show(3, truncate=False)


Ensamblaje exitoso. Vista previa del vector de entrada:
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                                                                                                                                                               

## **5.4 División en conjuntos de entrenamiento y prueba**

Una vez generada la muestra representativa `M` a partir de las particiones válidas `Mi` (seleccionadas con base en un umbral mínimo de 50,000 registros por combinación), se procede a dividir cada subconjunto `Mi` en dos partes:

- `Tri`: subconjunto de entrenamiento
- `Tsi`: subconjunto de prueba

Con el objetivo de mantener la representatividad y evitar sesgos, se implementa una estrategia de partición aleatoria estratificada, garantizando que:

- No exista traslape entre `Tri` y `Tsi`, es decir, `Tri ∩ Tsi = ∅`.
- La unión de todas las particiones (`Tri ∪ Tsi`) sea equivalente a la muestra total `M`.
- Se preserve la proporción original de la variable objetivo (`label`) dentro de ambos subconjuntos.

La proporción utilizada para la división es:

- **80% para el conjunto de entrenamiento (`Train`)**
- **20% para el conjunto de prueba (`Test`)**

A continuación, se muestra el código utilizado para realizar dicha partición.


In [ ]:
# Crear listas para almacenar subconjuntos de entrenamiento y prueba
train_subsets = []
test_subsets = []

# Para cada combinación válida, dividir en 80% Train y 20% Test
for fila in combinaciones_validas:
    p, h, v = fila["purpose"], fila["home_ownership"], fila["verification_status"]
    
    mi = M.filter(
        (col("purpose") == p) &
        (col("home_ownership") == h) &
        (col("verification_status") == v)
    )
    
    tri, tsi = mi.randomSplit([0.8, 0.2], seed=42)
    train_subsets.append(tri)
    test_subsets.append(tsi)

# Unir todos los subconjuntos para formar los conjuntos finales
train = train_subsets[0]
for t in train_subsets[1:]:
    train = train.union(t)

test = test_subsets[0]
for t in test_subsets[1:]:
    test = test.union(t)

# Mostrar resumen
print("Tamaño del conjunto de entrenamiento:", train.count())
print("Tamaño del conjunto de prueba:", test.count())

print("\nDistribución de clases en entrenamiento:")
train.groupBy("label").count().orderBy("label").show()

print("\nDistribución de clases en prueba:")
test.groupBy("label").count().orderBy("label").show()